## <center>Анализ аварий на ЖД транспорте США в 2013 году 
## <center>и страховых выплат по ним
<div style="border: 2px solid black; margin-left: 15%; margin-right: 15%"><img style="" src='https://www.atsb.gov.au/media/2444492/ro2010012_fig1.jpg' /></div>

### 1. Описание набора данных и признаков

В этом проекте исследуются данные об инцидентах грузового железнодорожного транспорта США за 2013 год и соответствующие запросы на страховое возмещение ущербра от перевозчиков. Данные взяты с <a href="http://www.explore-support.com/help/sample-data-sets">Cisco Data Explore</a>.

Датасет содержит следующие признаки:
<table class='desc_table' align='left' width='100%'>
<tr><th style="text-align: left !important">Признак</th><th style="text-align: left !important">Описание</th></tr>
<tr>
<td style="text-align: left !important">DEPARTURE CITY</td>
<td style="text-align: left !important">Город отправления груза (вагона)</td>
</tr>
<tr>
<td style="text-align: left !important">DEPARTURE STATE</td>
<td style="text-align: left !important">Штат отправления груза (вагона)</td>
</tr>
<tr>
<td style="text-align: left !important">DEPARTURE CARRIER</td>
<td style="text-align: left !important">Перевозчик отправления, отправитель</td>
</tr>
<tr>
<td style="text-align: left !important">ARRIVAL CITY</td>
<td style="text-align: left !important">Город прибытия груза (вагона)</td>
</tr>
<tr>
<td style="text-align: left !important">ARRIVAL STATE</td>
<td style="text-align: left !important">Штат прибытия груза (вагона)</td>
</tr>
<tr>
<td style="text-align: left !important">ARRIVAL CARRIER</td>
<td style="text-align: left !important">Компания-перевозчик прибытия, принимающая сторона</td>
</tr>
<tr>
<td style="text-align: left !important">RAIL SPEED SPEED</td>
<td style="text-align: left !important">Тип скороcти железной дороги</td>
</tr>
<tr>
<td style="text-align: left !important">RAIL CAR TYPE TYPE</td>
<td style="text-align: left !important">Тип вагона</td>
</tr>
<tr>
<td style="text-align: left !important">RAIL OWNERSHIP OWNERSHIP</td>
<td style="text-align: left !important">Тип собственности железной дороги</td>
</tr>
<tr>
<td style="text-align: left !important">RAIL CARLOAD LOAD</td>
<td style="text-align: left !important">Тип груза</td>
</tr>
<tr>
<td style="text-align: left !important">DEPEARTURE DATE</td>
<td style="text-align: left !important">Дата отправления</td>
</tr>
<tr>
<td style="text-align: left !important">ARRIVAL DATE</td>
<td style="text-align: left !important">Дата прибытия</td>
</tr>
<tr>
<td style="text-align: left !important">CAR VALUE</td>
<td style="text-align: left !important">Стоимость вагона, USD</td>
</tr>
<tr>
    <td style="text-align: left !important"><b>DAMAGED</b></td>
<td style="text-align: left !important"><b>Размер ущерба, USD</b></td>
</tr>
<tr>
<td style="text-align: left !important">WEIGHT</td>
<td style="text-align: left !important">Вес груза</td>
</tr>
<tr>
<td style="text-align: left !important">FUEL USED</td>
<td style="text-align: left !important">Количествто израсходованного топлива</td>
</tr>
<tr>
<td style="text-align: left !important">PROPER DESTINATION</td>
<td style="text-align: left !important">Метка правильности назначения</td>
</tr>
<tr>
<td style="text-align: left !important">MILES</td>
<td style="text-align: left !important">Пройденный путь</td>
</tr>
<tr>
<td style="text-align: left !important"># OF STOPS</td>
<td style="text-align: left !important">Количество остановок в пути</td>
</tr>
</table>

<b>Задача данного пректа</b> - попытаться предсказать размер ущерба, полученного в результате инцидента, а также предоставить другую полезную информацию. 

<b>Ценность результатов проекта</b> - информация для страховых компаний, позволяющая быть более гибкими в расчете стоимости страховки для тех или иных компаний, грузов, направлений и т. д., а также для самих перевозчиков - для прогноза затрат на перевозку грузов, выбора более безопасных путей, времени и других параметров для транспортировки грузов.

### 2. Первичный анализ данных

Импортируем все нужные библиотеки:

In [ ]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelBinarizer
from sklearn.model_selection import train_test_split, GridSearchCV, StratifiedKFold, cross_val_score

from sklearn.linear_model import LogisticRegression, LinearRegression
import xgboost as xgb

from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

import seaborn as sns

import matplotlib.pyplot as plt
%matplotlib inline

Теперь посмотрим на наши данные:

In [ ]:
data_file = '../../data/Rail_Insurance_Claims.csv'
data = pd.read_csv(data_file, sep=',', parse_dates=['DEPEARTURE DATE','ARRIVAL DATE'])

In [ ]:
data.head()

In [ ]:
data.info()

In [ ]:
data.describe()

In [ ]:
data.describe(include=['object'])

In [ ]:
data.describe(include='datetime64')

Видим, что в датасете большинство признаков - категориальные, целевая переменная и кол-во топлива - вещественные, два временных признака и несколько количественных. В данных нет пропусков. Также в данных есть много парных признаков DEPARTURE и ARRIVAL. Отсюда можно сделать следующие промежуточные выводы:
 * Категориальные признаки понадобится кодировать (OHE, mean target)
 * Из парных признаков можно извлечь много дополинтельной информации и создать новые признаки, связанные с путем следования, временем в пути и т.д.
 * В данных на первый взгляд нет пропусков - нужно проверить значения категориальных признаков на смысловые пропуски: значения типа N/A, unknown и т. д.
 * Из числовых признаков можно создать новые относительные.
 * Из временных признаков также можно извлечь дополнительную информацию
 * В данных присутствуют признаки CAR VALUE и DAMAGED - целесообразно предсказывать не сам ущерб, а его процент
 

In [ ]:
data['damaged percent'] = data['DAMAGED'] / data['CAR VALUE']
data['damaged percent'].describe()

Видим, что значения находятся в интервали от 0 до 1, т.е. признак корректный. Также можно заметить, что 75-я квантиль на порядок меньше максимального значения, из чего можно сделать вывод, что целевая переменная имееть сильный дисбаланс.

Посмотрим на распределения категориальных признаков и целевой переменной: построим несколько pivot-таблиц по категориальным признакам.

In [ ]:
(data.pivot_table( ['damaged percent'], ['DEPARTURE STATE'], ['ARRIVAL STATE'],  aggfunc='mean')) * 100

Посмотрим на распределение по штатам отправления:

In [ ]:
print(data['DEPARTURE STATE'].value_counts())

Создадим дополнительные временные признаки и посмотрим, возможно, есть перекос по дням или месяцам в ущербе для штата отправления TN:

In [ ]:
data['Dep Month'] = data['DEPEARTURE DATE'].dt.month
data['Dep Day'] = data['DEPEARTURE DATE'].dt.day

In [ ]:
(data[data['DEPARTURE STATE'] == 'TN'].pivot_table( ['damaged percent'], ['Dep Month'], ['Dep Day'], aggfunc='mean')) * 100

Продолжим с другими категориями:

In [ ]:
(data.pivot_table( ['damaged percent'], ['DEPARTURE CARRIER'], ['ARRIVAL CARRIER'],  aggfunc='mean')) * 100

In [ ]:
(data.pivot_table( ['damaged percent'], ['RAIL CARLOAD LOAD'], ['RAIL OWNERSHIP OWNERSHIP'],  aggfunc='mean')) * 100

In [ ]:
(data.pivot_table(['damaged percent'], ['RAIL CAR TYPE TYPE'], ['RAIL SPEED SPEED'], aggfunc='mean')) * 100

In [ ]:
(data.pivot_table(['damaged percent'], ['PROPER DESTINATION'], aggfunc='mean')) * 100

Построим корреляционную матрицу:

In [ ]:
corr_m = data.corr()

In [ ]:
round((corr_m), 2)

Проверим целевую переменную на нормальность и скошенность:

In [ ]:
from scipy.stats import shapiro, skewtest, skew

print('Normality: {}'.format(shapiro(data['damaged percent'])))
print('Skewness: {}'.format(skew(data['damaged percent'])))

Проверим, нет ли в категориальных признаках значений, похожих на пропуски:

In [ ]:
for c in data.select_dtypes(include=['object']):
    print('{}: {}'.format(c, data[c].unique()))

#### Выводы:
К предыдущим выводам можно добавить следующее:
* Вагоны, следующие из штата TN, терпят гораздо больший ущерб, чем из всех остальных штатов, но судя по распределению кол-ва рейсов из этого штата и средних потерь по датам, не похоже, что это выброс. Возможно, это будет хорошим предиктором.
* Перевозчики-отправители CN и  NS страдают немного сильнее, чем остальные, однако в ределах стандартного отклонения
* В распределении остальных категорий отностильено таргета ничего необычного не замечено
* С целевой переменной немного коррелируют стоимость вагона, вес груза, использованное топливо и кол-во остановок
* Целевая переменная не распределена нормально и имеет скошенность с тяжелым левым хвостом, понадобится ее преобразовать
* Выбросов и пропусков найти не удалось
* Значения переменной PROPER DESTINATION нужно преобразовать в [0, 1]

### 3. Первичный визуальный анализ данных

Для начала дополним данные недостающими временными признаками, преобразуем переменную PROPER DESTINATION:

In [ ]:
data['Dep DayOfWeek'] = data['DEPEARTURE DATE'].dt.weekday
data['Dep weekend'] = data['Dep DayOfWeek'].isin([5,6]).astype('int')

data['Arr Month'] = data['ARRIVAL DATE'].dt.month
data['Arr Day'] = data['ARRIVAL DATE'].dt.day
data['Arr DayOfWeek'] = data['ARRIVAL DATE'].dt.weekday
data['Arr weekend'] = data['Arr DayOfWeek'].isin([5,6]).astype('int')

data['proper_dest'] = data['PROPER DESTINATION'].map({'Yes': 1, 'No':0})

In [ ]:
data['duration'] = (data['ARRIVAL DATE'] - data['DEPEARTURE DATE']).dt.days

Отобразим корреляционную матрицу:

In [ ]:
c_m = data.corr()
plt.figure(figsize=(15, 15))
sns.heatmap(np.abs(c_m), annot=True, fmt=".2f", linewidths=.5)

Видна корреляция между днем отправления и днем прибытия, месяцем отправления и месяцем прибытия, что говорит о наличии расписания, корреляция между днем недели и выходным также ясна, как и между использованным топливом и весом груза. Интересна корреляция между правильным направлением и весом, нуждается в доп. изучении. Видно, что добавленные признаки немного коррелируют с целевой переменной. Переменную DAMAGED нужно удалить, она может быть получена из CAR VALUE и damaged percent.

In [ ]:
data.drop(['DAMAGED'], axis=1, inplace=True)

Отобразим плотности распределения числовых величин и колличество значений категориальных:

In [ ]:
clmns = data.select_dtypes(exclude=['object','datetime64', 'bool']).columns
f, axarr = plt.subplots(ncols=1, nrows=len(clmns), figsize=(15, 40))

for c in clmns:
    sns.distplot(data[c], ax=axarr[list(clmns).index(c)])

plt.tight_layout()

In [ ]:
categories = data.select_dtypes('object').columns

f, axarr = plt.subplots(ncols=1, nrows=len(categories), figsize=(15, 40))

i = 0
for cat in categories:
    g = sns.countplot(x=cat, data=data, ax=axarr[i], palette="Blues_d")
    
    r = 30
    if (cat.endswith('CITY')):
        r = 90
    g.set_xticklabels(g.get_xticklabels(), rotation=r)
    
    i += 1
plt.tight_layout(h_pad=0.5)
    

In [ ]:
categories = list(data.select_dtypes('object').columns) + ['# OF STOPS', 'Dep Month', 'Dep Day', 'Dep DayOfWeek', 'Arr Month', 'Arr Day', 'Arr DayOfWeek', 'Dep weekend', 'Arr weekend']

f, axarr = plt.subplots(ncols=1, nrows=len(categories), figsize=(15, 80))

i = 0
for cat in categories:
    g = sns.barplot(x=cat, y='damaged percent', data=data, ax=axarr[i])
    
    r = 30
    if (cat.endswith('CITY')):
        r = 90
    g.set_xticklabels(g.get_xticklabels(), rotation=r)
    
    i += 1
plt.tight_layout(h_pad=0.5)
    

In [ ]:
state_tbl = (data.pivot_table(['damaged percent'], ['DEPARTURE STATE'], ['ARRIVAL STATE'], aggfunc='mean')) * 100

plt.figure(figsize=(15, 15))
sns.heatmap(state_tbl)

#### Выводы
Видно, что результаты визуального анализа отображают закономерности, выявленные в предыдущей части. Распределения величин не указывает на наличие выбросов. Почти все значения признаков имеют разные средние значения damaged percent, т.е. должны быть учтены при прогнозе.

### 4. Инсайты и закономерности

Часть закономерностей описана выше.
Следует также создать признаки, связанные с путем, закодировать некоторые из них OHE, для путей как таковых применить mean target encoding.

### 5. Выбор метрики и модели

Т.к. в данных нет выбросов, а решаемая задача - регрессия, можно использовать MSE или R2. Т.к. R2 - это по сути 1 - усредненная MSE, то используем первую для большей наглядности. 

В качестве модели будем сравнивать линейную регрессию и градиентный бустинг, т.к. они обе подходят для задачи регрессии, а градиентный бустинг хорошо себя зарекомендовал в решениях задач со смешанным типом признаков. При использовании линейной регрессии будем масштабировать признаки.

### 6. Предобработка данных и создание новых признаков

Частично предобработка была выполнена выше для визуализации.

Создадим признаки, связанные с путем:

In [ ]:
data['interstate'] = (data['DEPARTURE STATE'] != data['ARRIVAL STATE']).astype('int')
data['intercity'] = (data['DEPARTURE CITY'] != data['ARRIVAL CITY']).astype('int')
data['intercarrier'] = (data['DEPARTURE CARRIER'] != data['ARRIVAL CARRIER']).astype('int')

In [ ]:
data['city route'] = data['DEPARTURE CITY'] + data['ARRIVAL CITY']
data['state route'] = data['DEPARTURE STATE'] + data['ARRIVAL STATE']
data['carrier route'] = data['DEPARTURE CARRIER'] + data['ARRIVAL CARRIER']

Создадим относительный признак расход доплива: 

In [ ]:
data['FUEL PER MILE'] = (data['FUEL USED']/data['MILES'])

Напишем функцию для кодирования средним:

In [ ]:
def mean_target_enc(train_df, y_train, valid_df, cat_features, skf):
    import warnings
    warnings.filterwarnings('ignore')
    target_name = y_train.name
    
    glob_mean = y_train.mean()
    train_df = pd.concat([train_df, pd.Series(y_train, name='y')], axis=1)
    new_train_df = train_df.copy()  

    for col in cat_features:
        new_train_df[col + '_mean_' + target_name] = [glob_mean for _ in range(new_train_df.shape[0])]

    for train_idx, valid_idx in skf.split(train_df, y_train):
        train_df_cv, valid_df_cv = train_df.iloc[train_idx, :], train_df.iloc[valid_idx, :]

        for col in cat_features:
            
            means = valid_df_cv[col].map(train_df_cv.groupby(col)['y'].mean())
            valid_df_cv[col + '_mean_' + target_name] = means.fillna(glob_mean)
            
        new_train_df.iloc[valid_idx] = valid_df_cv
    
    new_train_df.drop(['y'], axis=1, inplace=True)
    
    for col in cat_features:
        means = valid_df[col].map(train_df.groupby(col)['y'].mean())
        valid_df[col + '_mean_' + target_name] = means.fillna(glob_mean)
        
#     valid_df.drop(cat_features, axis=1, inplace=True)
    
    return new_train_df, valid_df

Применим OHE преобразование к категориальным признакам, за исключением путей:

In [ ]:
data_ohe = pd.get_dummies(data, columns=['DEPARTURE CITY', 'DEPARTURE STATE', 'DEPARTURE CARRIER',
       'ARRIVAL CITY', 'ARRIVAL STATE', 'ARRIVAL CARRIER', 'RAIL SPEED SPEED',
       'RAIL CAR TYPE TYPE', 'RAIL OWNERSHIP OWNERSHIP', 'RAIL CARLOAD LOAD'])

Применим mean target encoding к путям

In [ ]:
data_ohe, _ = mean_target_enc(data_ohe, (data['damaged percent']*10000).astype('int'), data_ohe[-1:], ['city route', 'state route', 'carrier route'], StratifiedKFold(5, shuffle=True, random_state=17))

Удалим ненужные фичи и выделим целевую переменную:

In [ ]:
y = data['damaged percent']
data_ohe.drop(['DEPEARTURE DATE', 'ARRIVAL DATE', 'FUEL USED', 'city route', 'state route', 'carrier route', 'PROPER DESTINATION'], axis=1, inplace=True)

In [ ]:
data_ohe.drop(['damaged percent'], axis=1, inplace=True)

In [ ]:

import scipy.stats as stats

stats.probplot(y, dist="norm", plot=plt)

In [ ]:

stats.probplot(np.log(y), dist="norm", plot=plt)

In [ ]:

stats.probplot(StandardScaler().fit_transform(y.values.reshape(-1,1).astype(np.float64)).flatten(), dist="norm", plot=plt)

In [ ]:
y = np.log(y)
y.hist()

Выделим тренировочную, валидационную и тестовые выборки. Т.к. данные у нас сбалансированы, выберем случайный способ. Отмасштабируем признаки.

In [ ]:

st = StandardScaler()

X_train, X_val, y_train, y_val = train_test_split(data_ohe, y, test_size=0.3, random_state=1)
X_train_st = st.fit_transform(X_train)
X_val_st = st.transform(X_val)

X_val_st, X_test_st, y_val, y_test = train_test_split(X_val_st, y_val, test_size=0.3333, random_state=1)

In [ ]:
lr = LinearRegression(n_jobs=-1)

cv_sc = cross_val_score(lr, X_train, y_train,  cv=5, n_jobs=-1)


In [ ]:
cv_sc

In [ ]:
lr.fit(X_train, y_train)
lr_pred_val = lr.predict(X_val_st)

r2_score(y_val, lr_pred_val)


In [ ]:
lr_test_pred = lr.predict((X_test_st))
print(r2_score(y_test, lr_test_pred))
np.sqrt(mean_squared_error(y_test, lr_test_pred))

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(np.exp(y_test).values[-200:], 'b')
plt.plot(np.exp(lr_test_pred)[-200:], 'g')

Видим, что линейная регрессия не сработала. Посмотрим на xboost.

In [ ]:


dtrain = xgb.DMatrix(X_train_st, label=np.sqrt(y_train))
dtest = xgb.DMatrix(X_val_st)

params = {
    'objective':'reg:linear',
    'max_depth':5,
    'silent':1,
    'nthread': 8,
#     'booster': 'dart',
#     'eta':0.5,
#     'gamma': 0.1,
#     'lambda': 20,
#     'alpha': 0.5
}

num_rounds = 100
xgb_ = xgb.train(params, dtrain, num_rounds)

xgb__pred = xgb_.predict(dtest)

r2_score(y_val, (xgb__pred))

In [ ]:

np.sqrt(mean_squared_error(np.sqrt(y_val), xgb__pred))

In [ ]:

np.sqrt(mean_squared_error(y_val, np.exp(xgb__pred)))

In [ ]:
xgb_test_pred = xgb_.predict(xgb.DMatrix(X_test_st))
print(r2_score(y_test, xgb_test_pred))
np.sqrt(mean_squared_error(y_test, xgb_test_pred))

In [ ]:
plt.figure(figsize=(15, 10))
plt.plot(np.exp(y_test).values[-200:], 'b')
plt.plot(np.exp(xgb_test_pred)[-200:], 'g')

А xgboost справляется неплохо, но предсказания получились смещенными вверх.

### Выводы:

Удалось проанализировать датасет с вполне приемлемыми результатами. Линейная регрессия плохо показала себя в этой задаче, а градиентный бустинг - довольно хорошо. 

Для улучшения результатов можно выделить больше признаков или закодировать средним больше категориальных фич. Также можно преобразовать задачу в бинарную классификацию, определяя обычные и большие убытки. 

Используя полученную модель, можно довольно хорошо предсказывать крупные потери при траспортировке, что будет полезно как страховым компаниям, так и перевозчикам.